## `Attribute Lookup Chain Review`

In [3]:
class Child:
    name = "Liam"

    def __init__(self, name):
        self.name = name

In [4]:
Child.__dict__

mappingproxy({'__module__': '__main__',
              'name': 'Liam',
              '__init__': <function __main__.Child.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Child' objects>,
              '__weakref__': <attribute '__weakref__' of 'Child' objects>,
              '__doc__': None})

In [5]:
c = Child("Anthony")

In [6]:
c.__dict__

{'name': 'Anthony'}

In [7]:
c.name

'Anthony'

In [8]:
class Child:
    name = "Liam"

    def __init__(self, name=None):
        if name:
            self.name = name


c = Child("Anthony")

In [9]:
c.name

'Anthony'

In [10]:
c2 = Child()

In [11]:
c2.name

'Liam'

In [23]:
class GrandParent(object):
    # name = "Robert"
    ...


class Parent(GrandParent):
    # name = "James"
    pass


class Child(Parent):
    # name = "Liam"

    def __init__(self, name=None):
        if name:
            self.name = name


c = Child()

In [22]:
c.name

AttributeError: AttributeError: 'Child' object has no attribute 'name'

In [ ]:
# __getattr__ implemented in object

## `The Descriptor Protocol`

In [24]:
class Descriptor:
    pass

In [25]:
# protcols -> contract between our objects and python

In [26]:
# the descriptor protocol:
# __get__()
# __set__()
# __delete__()

In [27]:
class Descriptor:
    def __get__(self, instance, owner):
        pass
    
    def __set__(self, instance, value):
        pass

    def __delete__(self, instance):
        pass

## `Using A Descriptor`

In [1]:
# SQL

In [2]:
# CREATE TABLE person (first_name varchar(200));

In [ ]:
# ORM -> OO vs SQL

In [ ]:
# target:
# > we want to be able to define a PersonTable calss that has a first_name attribute that is text of maximum length 200 

In [1]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __get__(self, instance, owner):
        return self.value
    

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        self.value = value

    def __delete__(self, instance):
        pass

In [2]:
class PersonTable:
    first_name = TextField(200)

In [3]:
p = PersonTable()

In [4]:
p.first_name = 'a' * 30

In [5]:
p.first_name

'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [6]:
# "binding beahvior  the precedence is given to the descriptor getter and setter methods over the instance attributes"

In [7]:
class PersonTable:
    first_name = TextField(200)

    def __init__(self, first_name):
        self.__dict__["first_name"] = first_name

In [8]:
p = PersonTable("Robbie")



In [9]:
p.first_name = "Liam"

In [10]:
p.__dict__

{'first_name': 'Robbie'}

In [11]:
p.first_name

'Liam'

## `Descriptor Storage`

In [1]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __get__(self, instance, owner):
        return self.value  

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")
            
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        self.value = value  
        
    def __delete__(self, instance):
        pass


class PersonTable:
    first_name = TextField(200)

In [2]:
p1 = PersonTable()
p2 = PersonTable()

In [3]:
p1.first_name = "Andrew"

In [4]:
p2.first_name

'Andrew'

In [7]:
class TextField:
    def __init__(self, length):
        self.length = length
        self._data = {}

    def __get__(self, instance, owner):
        # return self.value  
        return self._data.get(instance)

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")
            
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        # self.value = value
        self._data[instance] = value

        
    def __delete__(self, instance):
        pass


class PersonTable:
    first_name = TextField(200)

p1 = PersonTable()
p2 = PersonTable()

In [11]:
p1.first_name = "Andrew"
p2.first_name = "Bonnie"

In [13]:
p1.first_name, p2.first_name

('Andrew', 'Bonnie')

In [12]:
# "memory leak" 
#self._data[instance] = value ---hard reference---> instance
# this prevents the garbage collector to remove instances not being used, thus memory leak

sol -- use if instead dictionary

In [15]:
id(p1)

140194722656416

In [ ]:
# weakkey opposite of hard references

In [16]:
from weakref import WeakKeyDictionary

In [13]:
# descriptor -- use instance specific storage -- send data with field name where it should be stored
# instance.__dict__["fieldname "] direct storage

## `Even Better: Instance Storage`

In [27]:
class TextField:
    def __init__(self, length):
        self.length = length
        self._data = {}

    def __get__(self, instance, owner):
        # return self._data.get(instance)
        return instance.__dict__.get("text_field_value")

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        # self._data[instance] = value
        instance.__dict__["text_field_value"] = value

    def __delete__(self, instance):
        pass


class PersonTable:
    first_name = TextField(200)


p1 = PersonTable()
p2 = PersonTable()

p1.first_name = "Andrew"
p2.first_name = "Billy"

In [28]:
p1.first_name, p2.first_name 

('Andrew', 'Billy')

In [29]:
p1.__dict__, p2.__dict__

({'text_field_value': 'Andrew'}, {'text_field_value': 'Billy'})

In [30]:
class PersonTable:
    first_name = TextField(200)
    
    last_name = TextField(100)

In [31]:
p1 = PersonTable()

In [32]:
p1.first_name = "Andrew"
p1.last_name = "Greenfield"

In [34]:
p1.first_name, p1.last_name

('Greenfield', 'Greenfield')

In [35]:
p1.__dict__

{'text_field_value': 'Greenfield'}

In [39]:
class TextField:
    def __init__(self, length, field_name):
        self.length = length
        self.field_name = field_name

    def __get__(self, instance, owner):
        return instance.__dict__.get(f"TextField_{self.field_name}")

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        instance.__dict__[f"TextField_{self.field_name}"] = value

    def __delete__(self, instance):
        pass

In [42]:
class PersonTable:
    first_name = TextField(200, "first_name")
    
    last_name = TextField(100, "last_name")


p1 = PersonTable()

p1.first_name = "Andrew"
p1.last_name = "Greenfield"

p1.first_name, p1.last_name

('Andrew', 'Greenfield')

## `Using __set_name__`

In [14]:
# descriptor -- use instance specific storage -- send data with field name where it should be stored
# instance.__dict__["fieldname "] direct storage
# instead of field now use set name which will take the name of the field and store the value in the instance dict, no need to declare it separately

In [52]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        return instance.__dict__.get(f"TextField_{self.name}")

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        instance.__dict__[f"TextField_{self.name}"] = value

    def __delete__(self, instance):
        pass

In [53]:
class PersonTable:
    first_name = TextField(200)
    
    last_name = TextField(100)


p1 = PersonTable()

p1.first_name = "Andrew"
p1.last_name = "Greenfield"

p1.first_name, p1.last_name

('Andrew', 'Greenfield')

## `Tying Up Loose Ends`

In [18]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __set_name__(self, owner, name):
        # print(owner) - class name person table
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self

        return instance.__dict__.get(f"TextField_{self.name}")

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        instance.__dict__[f"TextField_{self.name}"] = value

    def __delete__(self, instance):
        del instance.__dict__[f"TextField_{self.name}"]

class PersonTable:
    first_name = TextField(200)    
    last_name = TextField(100)

In [60]:
PersonTable.__dict__

mappingproxy({'__module__': '__main__',
              'first_name': <__main__.TextField at 0x7f81a0fcc940>,
              'last_name': <__main__.TextField at 0x7f81a0be50a0>,
              '__dict__': <attribute '__dict__' of 'PersonTable' objects>,
              '__weakref__': <attribute '__weakref__' of 'PersonTable' objects>,
              '__doc__': None})

In [15]:
class NonPersonTable:
    first_name = TextField(200)    
    last_name = TextField(100)
#owners

In [70]:
PersonTable.first_name

In [83]:
p1 = PersonTable()
p1.first_name = "Andy"

In [84]:
p1.first_name

'Andy'

In [85]:
del p1.first_name

In [86]:
p1.first_name

## `Non-Data Descriptors`

In [19]:
# only getter method
#precedence comes after data decriptor, instance method

In [87]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self

        return instance.__dict__.get(f"TextField_{self.name}")

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        instance.__dict__[f"TextField_{self.name}"] = value

    def __delete__(self, instance):
        del instance.__dict__[f"TextField_{self.name}"]

In [126]:
from random import randint

class LuckyNumber:
    def __get__(self, instance, owner):
        return randint(1, 100)

    def __set__(self, instance, value):
        pass

In [134]:
class PersonTable:
    first_name = TextField(200)    
    last_name = TextField(100)
    personal_no = LuckyNumber()

    def __init__(self, personal_no):
        self.personal_no = personal_no


p = PersonTable(personal_no=10)

p.personal_no

74

In [124]:
PersonTable.personal_no

66

## `Aren't Properties Just Better?`

In [136]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self

        return instance.__dict__.get(f'TextField_{self.name}')

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        instance.__dict__[f'TextField_{self.name}'] = value

    def __delete__(self, instance):
        del instance.__dict__[f'TextField_{self.name}']


class PersonTableWithDescriptor:
    first_name = TextField(200)

In [145]:
class PersonTableWithProps:
    def __init__(self, first_name_length):
        self._TextField_first_name = None
        self.first_name_length = first_name_length

    @property
    def first_name(self):
        return self._TextField_first_name

    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")

        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name

In [147]:
p = PersonTableWithProps(200)

In [148]:
p.first_name = 2

TypeError: TypeError: Value should be a string

In [149]:
p.first_name = "a" * 2000

ValueError: ValueError: Value cannot exceed 200 characters

In [150]:
p.first_name = "Amadeus"

In [151]:
p.__dict__["first_name"] = "won't be able to get here..."

In [152]:
p.__dict__

{'_TextField_first_name': 'Amadeus',
 'first_name_length': 200,
 'first_name': "won't be able to get here..."}

In [153]:
p.first_name

'Amadeus'

In [154]:
# add 2 new fields:
# * last_name
# * occupation

In [20]:
#with prop, you have to repet all for each attribute unlike descriptor

In [ ]:
class PersonTableWithProps:
    def __init__(self, first_name_length, last_name_length, occupation_length):
        self._TextField_first_name = None
        self._TextField_last_name = None
        self._TextField_occupation = None

        self.first_name_length = first_name_length
        self.last_name_length = last_name_length
        self.occupation_length = occupation_length

    @property
    def first_name(self):
        return self._TextField_first_name

    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")

        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name

    @property
    def first_name(self):
        return self._TextField_first_name

    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")

        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name

    @property
    def first_name(self):
        return self._TextField_first_name

    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")

        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name

In [ ]:
class PersonTableWithDescriptor:
    first_name = TextField(200)
    last_name = TextField(50)
    occupation = TextField(100)

In [155]:
# mercedez benz is better than automobiles

In [156]:
# properties are better than descirptors

## `BONUS: Similar How?`

In [157]:
class TextField:
    def __init__(self, length):
        self.length = length

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self

        return instance.__dict__.get(f'TextField_{self.name}')

    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")

        instance.__dict__[f'TextField_{self.name}'] = value

    def __delete__(self, instance):
        del instance.__dict__[f'TextField_{self.name}']

In [159]:
class PersonTableWithProps:
    def __init__(self, first_name_length):
        self._TextField_first_name = None
        self.first_name_length = first_name_length

    def get_first_name(self):
        return self._TextField_first_name

    def set_first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")

        self._TextField_first_name = value
    
    def del_first_name(self):
        del self._TextField_first_name

    first_name = property(fget=get_first_name, fset=set_first_name, fdel=del_first_name)

In [160]:
class PersonTable:
    first_name = TextField(200)

In [161]:
# properties
# classmethod
# staticmethod
# __slots__

# ...all powered by descirptors under the hood.